# Call Fan Discovery

Build an optimized portfolio of BTO calls across multiple expirations for your conviction tickers.

**Steps:** Edit your tickers and budget below, then **Run All**.


In [ ]:
import math, os, warnings
from datetime import datetime, date

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    display_table,
    fetch_underlying_metrics,
    fetch_fan_candidates,
    score_option_candidates,
    build_best_fan,
    score_fan,
    bs_call_greeks,
    norm_cdf,
    build_correlation_matrix,
)
from notebook_reporting import export_report_bundle

warnings.filterwarnings("ignore", category=FutureWarning)
setup_report_style()

# ── Internal helpers (don't touch) ──────────────────────────────
FIGURE_COUNTER = 0


def _show(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    fig.update_layout(title=f"Figure {FIGURE_COUNTER}. {title}")
    fig.show()


def _pop(spot, be, dte, iv, r=0.04):
    """P(S_T > breakeven) via log-normal model."""
    if dte <= 0 or iv <= 0 or spot <= 0 or be <= 0:
        return 0.0
    t = dte / 365.0
    return norm_cdf((math.log(spot / be) + (r - 0.5 * iv**2) * t) / (iv * math.sqrt(t)))


def _bucket(dte):
    if dte <= 30:
        return "Short (< 30d)"
    if dte <= 120:
        return "Medium (1-4mo)"
    return "Long (4mo+)"


def _warn(r):
    w = []
    if r["iv"] > r["hv"] * 1.5:
        w.append("IV rich")
    if r["iv"] > 0.60:
        w.append("IV > 60%")
    if r["spread_pct"] > 0.15:
        w.append("Wide bid/ask")
    if r["oi"] < 50:
        w.append("Low OI")
    if r["dte"] <= 7:
        w.append("EXPIRING")
    elif r["dte"] <= 14:
        w.append("< 2 wk")
    if r["be_pct"] > 0.15:
        w.append(f"BE +{r['be_pct']:.0%}")
    if r["rsi"] > 70:
        w.append("Overbought")
    if r["rsi"] < 30:
        w.append("Oversold")
    if r["ret_3m"] < -0.10:
        w.append("Downtrend")
    return ", ".join(w) if w else "--"

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║           EDIT HERE — tickers, strike zones, budget         ║
# ╚══════════════════════════════════════════════════════════════╝

MY_TICKERS = {
    "CMI": (500, 650),  # (low_strike, high_strike) or None for auto
    "SNPS": (415, 560),
    "GEV": (750, 840),
    "AVGO": (370, 460),
    "KLAC": (1300, 1600),
    "WDC": (280, 300),
    "TSM": (340, 420),
    "MU": (370, 390),
}

MY_BUDGET = 15_000  # total cash to deploy

MAX_LEGS = 8  # max expirations per ticker

# ── Profit Filters (auto-drop bad setups) ────────────────────────
# Ticker-level gates — remove names not worth buying calls on
MIN_RET_3M = -0.15  # drop tickers down > 15% over 3 months
MAX_RSI = 80  # drop severely overbought entries
MIN_FAN_SCORE = 35  # drop fans scoring below this

# Leg-level gates — skip bad risk/reward individual contracts
MIN_POP = 0.15  # need >= 15% probability of profit
MAX_BE_DIST = 0.20  # skip if breakeven needs > 20% stock move
MAX_IV_HV = 1.80  # skip IV-rich legs (IV > 1.8x realized vol)
MIN_OI = 20  # skip illiquid contracts
MAX_SPREAD = 0.15  # skip wide bid-ask spreads
MAX_THETA_DRAIN = 0.03  # skip if daily theta > 3% of premium

# ── Internals (no need to edit) ─────────────────────────────────
assert MY_BUDGET > 0, "MY_BUDGET must be positive"
assert len(MY_TICKERS) > 0, "Add at least one ticker"
for _t, _r in MY_TICKERS.items():
    if _r is not None:
        assert len(_r) == 2 and _r[0] < _r[1], f"{_t}: strike range must be (low, high)"

STARTING_BALANCE = float(MY_BUDGET)
FAN_TICKERS = MY_TICKERS
MAX_LEGS_PER_FAN = MAX_LEGS
MIN_DTE = 3
MAX_DTE = 760
DEFAULT_MONEYNESS = (0.85, 1.10)
RATE_LIMIT_SLEEP = 0.25
EXPORT_EXCEL = True
EXPORT_ZIP = True
RUN_STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = "outputs"

display(
    Markdown(
        f"**Budget:** ${STARTING_BALANCE:,.0f}  |  "
        f"**Tickers:** {', '.join(FAN_TICKERS)}  |  "
        f"**Max legs/fan:** {MAX_LEGS_PER_FAN}"
    )
)

**Budget:** $15,000  |  **Tickers:** CMI, SNPS, GEV, AVGO, KLAC, WDC, TSM, MU  |  **Max legs/fan:** 8

In [ ]:
# ── Fetch market data ────────────────────────────────────────────
tickers = list(FAN_TICKERS.keys())

display(Markdown("Fetching underlying data ..."))
metrics_df = fetch_underlying_metrics(
    tickers, history_period="1y", rate_limit_sleep=RATE_LIMIT_SLEEP
)
assert not metrics_df.empty, "Could not fetch data for any ticker."

for _, row in metrics_df.iterrows():
    rng = FAN_TICKERS.get(row["ticker"])
    zone = f"${rng[0]:,.0f}-${rng[1]:,.0f}" if rng else "auto"
    display(
        Markdown(
            f"**{row['ticker']}** ${row['spot']:,.2f}  |  zone {zone}  |  "
            f"3m {row['ret_3m']:+.0%}  |  HV {row['hv_30']:.0%}  |  "
            f"RSI {row['rsi_14']:.0f}  |  Beta {row['beta']:.2f}"
        )
    )

display(Markdown("---\nFetching option chains ..."))
fan_chains: dict[str, pd.DataFrame] = {}
for tkr, strike_rng in FAN_TICKERS.items():
    row = metrics_df[metrics_df["ticker"] == tkr]
    if row.empty:
        continue
    spot = float(row.iloc[0]["spot"])
    df = fetch_fan_candidates(
        tkr,
        spot=spot,
        strike_range=strike_rng,
        moneyness_range=DEFAULT_MONEYNESS,
        min_dte=MIN_DTE,
        max_dte=MAX_DTE,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    if df.empty:
        display(Markdown(f"**{tkr}**: no contracts in zone"))
    else:
        display(
            Markdown(
                f"**{tkr}**: {len(df)} contracts, "
                f"{df['expiration'].nunique()} expirations"
            )
        )
        fan_chains[tkr] = df

assert fan_chains, "No option chains fetched. Check tickers and strike zones."
display(Markdown("**Done.**"))

Fetching underlying data ...

**CMI** $577.73  |  zone $500-$650  |  3m +32%  |  HV 45%  |  RSI 50  |  Beta 1.12

**SNPS** $426.88  |  zone $415-$560  |  3m +4%  |  HV 44%  |  RSI 23  |  Beta 1.13

**GEV** $779.35  |  zone $750-$840  |  3m +39%  |  HV 43%  |  RSI 71  |  Beta nan

**AVGO** $332.92  |  zone $370-$460  |  3m -7%  |  HV 39%  |  RSI 40  |  Beta 1.22

...

**GEV**: 147 contracts, 15 expirations

**AVGO**: 212 contracts, 23 expirations

**KLAC**: 138 contracts, 10 expirations

**WDC**: 51 contracts, 13 expirations

**TSM**: 179 contracts, 17 expirations

In [ ]:
# ══════════════════════════════════════════════════════════════════
#  OPTIMIZER — scores every candidate, filters junk, fits to budget
# ══════════════════════════════════════════════════════════════════

# 1. Build best fan per ticker
fans: dict[str, pd.DataFrame] = {}
fan_metrics: dict[str, dict] = {}
for tkr, chain_df in fan_chains.items():
    tkr_m = metrics_df[metrics_df["ticker"] == tkr]
    spot = float(tkr_m.iloc[0]["spot"])
    scored = score_option_candidates(chain_df, tkr_m)
    if scored.empty:
        continue
    fan = build_best_fan(scored, max_legs=MAX_LEGS_PER_FAN)
    if fan.empty:
        continue
    fans[tkr] = fan
    fan_metrics[tkr] = score_fan(fan, spot)

# ── TICKER QUALITY GATE ─────────────────────────────────────────
# Auto-drop tickers that fail profitability checks so we don't
# burn budget on bad setups.
dropped_tickers: dict[str, list[str]] = {}
for tkr in list(fans.keys()):
    m = metrics_df[metrics_df["ticker"] == tkr].iloc[0]
    reasons = []
    ret3 = float(m["ret_3m"])
    rsi = float(m["rsi_14"])
    hv_v = float(m["hv_30"]) if float(m["hv_30"]) > 0 else 0.28
    avg_fan_iv = float(fans[tkr]["iv"].mean())
    if ret3 < MIN_RET_3M:
        reasons.append(f"downtrend: 3m return {ret3:+.0%}")
    if rsi > MAX_RSI:
        reasons.append(f"overbought: RSI {rsi:.0f}")
    fm = fan_metrics.get(tkr, {})
    if fm and fm.get("fan_score", 100) < MIN_FAN_SCORE:
        reasons.append(f"weak fan: {fm['fan_score']:.1f}")
    if hv_v > 0 and avg_fan_iv / hv_v > MAX_IV_HV + 0.3:
        reasons.append(f"IV/HV {avg_fan_iv / hv_v:.1f}x across fan")
    # If even a +10% bull move loses money, the setup is bad
    if fm and fm.get("scenario_pnl", {}).get("bull_10", 0) <= 0:
        reasons.append("loses money on +10% move")
    if reasons:
        dropped_tickers[tkr] = reasons
        del fans[tkr]
        del fan_metrics[tkr]

if dropped_tickers:
    display(Markdown("### Tickers Removed"))
    for tkr, reasons in dropped_tickers.items():
        display(Markdown(f"- **{tkr}** -- {'; '.join(reasons)}"))
    display(Markdown(""))

# Fan-level summary (kept for deep-dive charts)
fan_summary_df = (
    pd.DataFrame(
        [
            {
                "ticker": t,
                "fan_score": fm["fan_score"],
                "legs": fm["n_legs"],
                "total_cost": fm["total_cost_100sh"],
                "agg_delta": fm["agg_delta"],
                "agg_theta": fm["agg_theta"],
                "agg_vega": fm["agg_vega"],
                "total_premium": fm["total_premium"],
                "iv_term_slope": fm["iv_term_slope"],
            }
            for t, fm in fan_metrics.items()
        ]
    ).sort_values("fan_score", ascending=False)
    if fan_metrics
    else pd.DataFrame()
)

# 2. Pool candidate legs — apply per-leg profit filters
pool_rows = []
dropped_legs = 0
leg_drop_reasons: dict[str, int] = {}

for tkr, fan in fans.items():
    m = metrics_df[metrics_df["ticker"] == tkr].iloc[0]
    spot = float(m["spot"])
    beta = max(float(m["beta"]) if not np.isnan(m["beta"]) else 1.0, 0.3)
    hv = float(m["hv_30"]) if float(m["hv_30"]) > 0 else 0.28
    rsi_v = float(m["rsi_14"])
    ret3m = float(m["ret_3m"])
    for _, lg in fan.iterrows():
        mid = float(lg["mid"])
        strike = float(lg["strike"])
        dte = int(lg["dte"])
        iv = float(lg["iv"])
        oi = float(lg["open_interest"])
        sp_pct = float(lg["spread_pct"]) if not np.isnan(lg["spread_pct"]) else 0
        cost = mid * 100
        g = bs_call_greeks(spot=spot, strike=strike, dte=dte, iv=iv)
        be = strike + mid
        be_pct = be / spot - 1
        prob = _pop(spot, be, dte, iv)
        sc = float(lg["master_score"]) if "master_score" in lg.index else 50.0
        iv_hv = iv / hv if hv > 0 else 99.0

        # ── Leg sanity filters ──────────────────────────────────
        skip = []
        if prob < MIN_POP:
            skip.append("Low PoP")
        if be_pct > MAX_BE_DIST:
            skip.append("BE too far")
        if iv_hv > MAX_IV_HV:
            skip.append("IV rich")
        if oi < MIN_OI:
            skip.append("Low OI")
        if sp_pct > MAX_SPREAD:
            skip.append("Wide spread")
        if (
            mid > 0
            and not np.isnan(g["theta"])
            and abs(g["theta"]) / mid > MAX_THETA_DRAIN
        ):
            skip.append("Theta drain")
        if skip:
            dropped_legs += 1
            for r in skip:
                leg_drop_reasons[r] = leg_drop_reasons.get(r, 0) + 1
            continue

        # ── Risk-adjusted efficiency ────────────────────────────
        # Penalize IV richness: paying 1.5x HV is ok, 1.8x gets discounted
        iv_discount = max(1.0 - (iv_hv - 1.0) * 0.3, 0.3)
        # Penalize fast theta decay relative to premium paid
        theta_ratio = (
            abs(g["theta"]) / mid if mid > 0 and not np.isnan(g["theta"]) else 0
        )
        theta_eff = max(1.0 - theta_ratio * 10, 0.2)
        eff = (sc * prob * iv_discount * theta_eff) / (cost * beta) if cost > 0 else 0

        pool_rows.append(
            {
                "ticker": tkr,
                "contract": lg["contract_symbol"],
                "strike": strike,
                "expiration": lg["expiration"],
                "dte": dte,
                "bucket": _bucket(dte),
                "mid": mid,
                "cost_1ct": cost,
                "iv": iv,
                "oi": oi,
                "spread_pct": sp_pct,
                "iv_hv": iv_hv,
                "delta": g["delta"],
                "theta": g["theta"],
                "vega": g["vega"],
                "breakeven": be,
                "be_pct": be_pct,
                "pop": prob,
                "score": sc,
                "beta": beta,
                "efficiency": eff,
                "hv": hv,
                "rsi": rsi_v,
                "ret_3m": ret3m,
            }
        )

if dropped_legs:
    reasons_str = ", ".join(
        f"{r}: {n}" for r, n in sorted(leg_drop_reasons.items(), key=lambda x: -x[1])
    )
    display(Markdown(f"Filtered **{dropped_legs}** legs ({reasons_str})"))

pool = pd.DataFrame(pool_rows).sort_values("efficiency", ascending=False)

# 3. Greedy knapsack — fit to budget
#    Per-ticker cap = 50% of budget.  Qty = 1 contract each.
TICKER_CAP = 0.50
selected, budget_left = [], STARTING_BALANCE
tkr_spent: dict[str, float] = {}
cap_dollars = STARTING_BALANCE * TICKER_CAP

for _, lg in pool.iterrows():
    c, t = lg["cost_1ct"], lg["ticker"]
    if c > budget_left or (tkr_spent.get(t, 0) + c) > cap_dollars:
        continue
    selected.append(lg)
    budget_left -= c
    tkr_spent[t] = tkr_spent.get(t, 0) + c

buy_df = pd.DataFrame(selected).reset_index(drop=True)
buy_df.insert(0, "#", range(1, len(buy_df) + 1))

# ── Hard budget guarantee ───────────────────────────────────────
total_deployed = buy_df["cost_1ct"].sum() if not buy_df.empty else 0
assert total_deployed <= STARTING_BALANCE, (
    f"BUG: deployed ${total_deployed:,.0f} exceeds budget ${STARTING_BALANCE:,.0f}"
)
cash_left = STARTING_BALANCE - total_deployed
utilization = total_deployed / STARTING_BALANCE if STARTING_BALANCE > 0 else 0

# ══════════════════════════════════════════════════════════════════
#                        B U Y   L I S T
# ══════════════════════════════════════════════════════════════════
if buy_df.empty:
    display(
        Markdown(
            "## No positions fit the budget.\n\n"
            "Try: increase `MY_BUDGET`, widen strike zones, loosen filters, "
            "or add more tickers."
        )
    )
else:
    # Enrich
    buy_df["stop_loss"] = buy_df["mid"] * 0.50
    buy_df["sell_50"] = buy_df["mid"] * 1.50
    buy_df["sell_100"] = buy_df["mid"] * 2.00
    buy_df["stock_at_50"] = buy_df["strike"] + buy_df["sell_50"]
    buy_df["stock_at_100"] = buy_df["strike"] + buy_df["sell_100"]
    buy_df["flags"] = buy_df.apply(_warn, axis=1)

    # ── Header ──────────────────────────────────────────────────
    display(Markdown("---"))
    display(Markdown("# BUY LIST"))
    display(Markdown(f"*Generated {datetime.now().strftime('%b %d, %Y %H:%M')}*"))

    # Budget bar
    bar_filled = int(utilization * 30)
    bar_empty = 30 - bar_filled
    bar = f"`[{'█' * bar_filled}{'░' * bar_empty}]` {utilization:.0%}"
    display(
        Markdown(
            f"| | |\n|---|---|\n"
            f"| **Budget** | ${STARTING_BALANCE:,.0f} |\n"
            f"| **Deploying** | ${total_deployed:,.0f} |\n"
            f"| **Cash reserve** | ${cash_left:,.0f} |\n"
            f"| **Utilization** | {bar} |\n"
            f"| **Positions** | {len(buy_df)} contracts across "
            f"{buy_df['ticker'].nunique()} tickers |"
        )
    )

    # ── Transaction table ───────────────────────────────────────
    display(Markdown("---"))
    display_table(
        buy_df[
            [
                "#",
                "ticker",
                "contract",
                "strike",
                "expiration",
                "dte",
                "bucket",
                "mid",
                "cost_1ct",
                "iv",
                "iv_hv",
                "delta",
                "pop",
                "flags",
            ]
        ].rename(
            columns={
                "ticker": "Ticker",
                "contract": "Contract",
                "strike": "Strike",
                "expiration": "Exp",
                "dte": "DTE",
                "bucket": "Timeline",
                "mid": "Price",
                "cost_1ct": "Cost",
                "iv": "IV",
                "iv_hv": "IV/HV",
                "delta": "Delta",
                "pop": "PoP",
                "flags": "Flags",
            }
        ),
        caption="Buy to Open (1 contract each)",
        format_dict={
            "Strike": "${:,.0f}",
            "Price": "${:,.2f}",
            "Cost": "${:,.0f}",
            "IV": "{:.0%}",
            "IV/HV": "{:.1f}x",
            "Delta": "{:.2f}",
            "PoP": "{:.0%}",
        },
    )

    # ── Exit plan ───────────────────────────────────────────────
    display(Markdown("---"))
    display(Markdown("## Exit Plan"))
    display_table(
        buy_df[
            [
                "#",
                "ticker",
                "contract",
                "mid",
                "breakeven",
                "be_pct",
                "stop_loss",
                "sell_50",
                "sell_100",
            ]
        ].rename(
            columns={
                "ticker": "Ticker",
                "contract": "Contract",
                "mid": "Paid",
                "breakeven": "BE",
                "be_pct": "BE Dist",
                "stop_loss": "Stop -50%",
                "sell_50": "TP +50%",
                "sell_100": "TP +100%",
            }
        ),
        caption="Sell Targets",
        format_dict={
            "Paid": "${:,.2f}",
            "BE": "${:,.0f}",
            "BE Dist": "{:+.0%}",
            "Stop -50%": "${:,.2f}",
            "TP +50%": "${:,.2f}",
            "TP +100%": "${:,.2f}",
        },
    )
    display(
        Markdown(
            "**BE** = breakeven stock price at expiry  |  "
            "**Stop** = sell if option drops to this  |  "
            "**TP** = take-profit on the option price"
        )
    )

    # ── Portfolio blend ─────────────────────────────────────────
    display(Markdown("---"))
    display(Markdown("## Portfolio"))

    # By ticker
    tkr_b = (
        buy_df.groupby("ticker")
        .agg(
            legs=("contract", "count"),
            cost=("cost_1ct", "sum"),
            avg_pop=("pop", "mean"),
            avg_delta=("delta", "mean"),
            beta=("beta", "first"),
        )
        .reset_index()
    )
    tkr_b["wt"] = tkr_b["cost"] / total_deployed
    tkr_b["beta_c"] = tkr_b["wt"] * tkr_b["beta"]
    tkr_b = tkr_b.sort_values("cost", ascending=False)

    display_table(
        tkr_b.rename(
            columns={
                "ticker": "Ticker",
                "legs": "Legs",
                "cost": "Deployed",
                "wt": "Weight",
                "beta": "Beta",
                "beta_c": "Beta Contrib",
                "avg_pop": "Avg PoP",
                "avg_delta": "Avg Delta",
            }
        ),
        caption="By Ticker",
        format_dict={
            "Deployed": "${:,.0f}",
            "Weight": "{:.0%}",
            "Beta": "{:.2f}",
            "Beta Contrib": "{:.2f}",
            "Avg PoP": "{:.0%}",
            "Avg Delta": "{:.2f}",
        },
    )

    # By timeline
    tm_b = (
        buy_df.groupby("bucket")
        .agg(
            legs=("contract", "count"),
            cost=("cost_1ct", "sum"),
            avg_pop=("pop", "mean"),
            avg_dte=("dte", "mean"),
        )
        .reset_index()
    )
    tm_b["wt"] = tm_b["cost"] / total_deployed
    tm_b = tm_b.sort_values("avg_dte")

    display_table(
        tm_b.rename(
            columns={
                "bucket": "Timeline",
                "legs": "Legs",
                "cost": "Deployed",
                "wt": "Weight",
                "avg_pop": "Avg PoP",
                "avg_dte": "Avg DTE",
            }
        ),
        caption="By Timeline",
        format_dict={
            "Deployed": "${:,.0f}",
            "Weight": "{:.0%}",
            "Avg PoP": "{:.0%}",
            "Avg DTE": "{:.0f}",
        },
    )

    port_beta = tkr_b["beta_c"].sum()
    port_pop = buy_df["pop"].mean()
    port_delta = buy_df["delta"].sum()
    port_theta = buy_df["theta"].sum()

    display(
        Markdown(
            f"**Portfolio beta:** {port_beta:.2f}  |  "
            f"**Avg PoP:** {port_pop:.0%}  |  "
            f"**Delta:** {port_delta:.2f}  |  "
            f"**Theta/day:** ${port_theta:+.2f}"
        )
    )

    # ── Position rules ──────────────────────────────────────────
    display(Markdown("---"))
    display(Markdown("## Rules"))
    display(
        Markdown(
            "| Trigger | Action |\n|---|---|\n"
            "| Option down 50% | **Close.** Redeploy |\n"
            "| Option up 50% | **Sell a third**, move stop to breakeven |\n"
            "| Option up 100% | **Sell half.** Rest rides free |\n"
            "| DTE < 21 days | **Roll or close** (theta cliff) |\n"
            "| DTE < 7 days | **Close** unless deep ITM |\n"
            "| Stock gaps > 5% | Re-evaluate thesis |\n"
            "| IV moves > 20% | Reassess: sell into spike, tighten on crush |\n"
            f"| Portfolio -${STARTING_BALANCE * 0.10:,.0f} | **Pause.** Reassess |\n"
            f"| Portfolio +${STARTING_BALANCE * 0.30:,.0f} | **Trim.** Lock gains |"
        )
    )

Filtered **8** legs (Theta drain: 5, Low OI: 3, Wide spread: 2)

---

# BUY LIST

*Generated Feb 08, 2026 18:12*

| | |
|---|---|
| **Budget** | $15,000 |
| **Deploying** | $13,725 |
| **Cash reserve** | $1,275 |
| **Utilization** | `[███████████████████████████░░░]` 92% |
| **Positions** | 4 contracts across 3 tickers |

...

,Timeline,Legs,Deployed,Avg PoP,Avg DTE,Weight
1,Short (< 30d),2,"$7,940",36%,19,58%
0,Medium (1-4mo),2,"$5,785",34%,40,42%


**Portfolio beta:** 1.08  |  **Avg PoP:** 35%  |  **Delta:** 2.32  |  **Theta/day:** $-2.37

---

## Rules

| Trigger | Action |
|---|---|
| Option down 50% | **Close.** Redeploy |
| Option up 50% | **Sell a third**, move stop to breakeven |
| Option up 100% | **Sell half.** Rest rides free |
| DTE < 21 days | **Roll or close** (theta cliff) |
| DTE < 7 days | **Close** unless deep ITM |
| Stock gaps > 5% | Re-evaluate thesis |
| IV moves > 20% | Reassess: sell into spike, tighten on crush |
| Portfolio -$1,500 | **Pause.** Reassess |
| Portfolio +$4,500 | **Trim.** Lock gains |

---

## Deep Dive (optional)

Charts and scenario analysis below. Scroll past if you just need the buy list above.


In [ ]:
# --- Key Charts ---
if fans:
    all_fans = pd.concat(fans.values(), ignore_index=True)

    # Premium Ladder
    fig_prem = px.scatter(
        all_fans,
        x="dte",
        y="mid",
        color="ticker",
        size="open_interest",
        hover_data=["contract_symbol", "strike", "iv", "expiration"],
        height=450,
    )
    fig_prem.update_yaxes(title="Premium ($)", tickprefix="$")
    fig_prem.update_xaxes(title="Days to Expiration")
    _show(fig_prem, "Premium vs DTE")

    # IV Term Structure
    fig_iv = px.line(
        all_fans,
        x="dte",
        y="iv",
        color="ticker",
        markers=True,
        height=400,
    )
    fig_iv.update_yaxes(tickformat=".0%", title="Implied Volatility")
    fig_iv.update_xaxes(title="Days to Expiration")
    _show(fig_iv, "IV Term Structure")

    # Fan Score Ranking
    fig_rank = px.bar(
        fan_summary_df.sort_values("fan_score"),
        x="fan_score",
        y="ticker",
        orientation="h",
        color="fan_score",
        color_continuous_scale="Blues",
        height=max(250, len(fan_summary_df) * 70),
        text="fan_score",
    )
    fig_rank.update_traces(texttemplate="%{text:.1f}", textposition="outside")
    fig_rank.update_layout(coloraxis_showscale=False)
    fig_rank.update_xaxes(title="Fan Score")
    _show(fig_rank, "Fan Score Ranking")

In [ ]:
# --- What-If Analysis ---
if fan_metrics:
    scen_rows = []
    for tkr, fm in fan_metrics.items():
        for label, pnl in fm["scenario_pnl"].items():
            clean = (
                label.replace("_", " ")
                .replace("bear", "Bear")
                .replace("bull", "Bull")
                .replace("flat", "Flat")
            )
            scen_rows.append({"ticker": tkr, "scenario": clean, "pnl": pnl})

    fig_scen = px.bar(
        pd.DataFrame(scen_rows),
        x="ticker",
        y="pnl",
        color="scenario",
        barmode="group",
        height=450,
        color_discrete_sequence=["#B0533C", "#D4A574", "#8B9BB4", "#4C6E91", "#1F3A5F"],
    )
    fig_scen.update_yaxes(title="P&L per set ($)", tickprefix="$")
    _show(fig_scen, "Scenario P&L (per 1-contract set)")

    # Sensitivity heatmap for top fan
    top_tkr = fan_summary_df.iloc[0]["ticker"]
    top_fan = fans[top_tkr]
    top_spot = float(metrics_df[metrics_df["ticker"] == top_tkr].iloc[0]["spot"])

    spot_moves = np.arange(-0.25, 0.30, 0.05)
    iv_shocks = np.arange(-0.15, 0.20, 0.05)
    surface = np.zeros((len(iv_shocks), len(spot_moves)))

    for i, iv_shock in enumerate(iv_shocks):
        for j, spot_move in enumerate(spot_moves):
            total_pnl = 0.0
            for _, leg in top_fan.iterrows():
                terminal = top_spot * (1.0 + spot_move)
                intrinsic = max(terminal - float(leg["strike"]), 0.0)
                new_iv = max(float(leg["iv"]) + iv_shock, 0.05)
                half_dte = max(int(leg["dte"]) // 2, 1)
                if intrinsic > 0 and half_dte > 1:
                    g = bs_call_greeks(
                        spot=terminal,
                        strike=float(leg["strike"]),
                        dte=half_dte,
                        iv=new_iv,
                    )
                    time_val = (
                        max(g["vega"] * new_iv * 10, 0)
                        if not np.isnan(g["vega"])
                        else 0
                    )
                    leg_val = intrinsic + time_val
                else:
                    leg_val = intrinsic
                total_pnl += leg_val - float(leg["mid"])
            surface[i, j] = total_pnl

    fig_surf = go.Figure(
        data=go.Heatmap(
            z=surface,
            x=[f"{m:+.0%}" for m in spot_moves],
            y=[f"{s:+.0%}" for s in iv_shocks],
            colorscale="RdYlGn",
            text=np.round(surface, 1).astype(str),
            texttemplate="$%{text}",
            hovertemplate="Spot: %{x}<br>IV shock: %{y}<br>P&L: $%{z:,.0f}<extra></extra>",
        )
    )
    fig_surf.update_layout(
        height=450, xaxis_title="Stock Move", yaxis_title="IV Change"
    )
    _show(fig_surf, f"{top_tkr} What-If Surface (Stock Move x IV Change)")

In [ ]:
# --- Portfolio Risk Summary ---
if not buy_df.empty:
    total_deployed = buy_df["cost_1ct"].sum()
    remaining = STARTING_BALANCE - total_deployed
    total_theta = buy_df["theta"].sum()
    total_delta = buy_df["delta"].sum()
    total_vega = buy_df["vega"].sum()
    weekly_theta = total_theta * 7

    # Allocation donut — by ticker
    tkr_costs = buy_df.groupby("ticker")["cost_1ct"].sum().reset_index()
    fig_donut = go.Figure(
        data=[
            go.Pie(
                labels=tkr_costs["ticker"],
                values=tkr_costs["cost_1ct"],
                hole=0.50,
                marker_colors=[
                    "#1F3A5F",
                    "#4C6E91",
                    "#B0533C",
                    "#D4A574",
                    "#6B8E6B",
                    "#8B9BB4",
                ],
                textinfo="label+percent",
            )
        ]
    )
    fig_donut.update_layout(
        height=380,
        annotations=[
            dict(
                text=f"${total_deployed:,.0f}",
                x=0.5,
                y=0.5,
                font_size=18,
                showarrow=False,
            ),
        ],
    )
    _show(fig_donut, "Capital Allocation by Ticker")

    # Timeline donut
    time_costs = buy_df.groupby("bucket")["cost_1ct"].sum().reset_index()
    fig_time = go.Figure(
        data=[
            go.Pie(
                labels=time_costs["bucket"],
                values=time_costs["cost_1ct"],
                hole=0.50,
                marker_colors=["#B0533C", "#4C6E91", "#1F3A5F"],
                textinfo="label+percent",
            )
        ]
    )
    fig_time.update_layout(
        height=380,
        annotations=[
            dict(
                text=f"{len(buy_df)} legs", x=0.5, y=0.5, font_size=16, showarrow=False
            ),
        ],
    )
    _show(fig_time, "Capital Allocation by Timeline")

    # Risk summary
    port_beta = (
        buy_df.groupby("ticker")
        .agg(cost=("cost_1ct", "sum"), beta=("beta", "first"))
        .assign(w=lambda d: d["cost"] / total_deployed)
        .eval("bc = w * beta")["bc"]
        .sum()
    )

    display(Markdown("### Risk at a Glance"))
    display(
        Markdown(
            f"| Metric | Value |\n|---|---|\n"
            f"| Deployed | ${total_deployed:,.0f} of ${STARTING_BALANCE:,.0f} ({total_deployed / STARTING_BALANCE:.0%}) |\n"
            f"| Cash reserve | ${remaining:,.0f} |\n"
            f"| **Max loss** | **${total_deployed:,.0f}** (all premiums) |\n"
            f"| Portfolio beta | {port_beta:.2f} |\n"
            f"| Portfolio delta | {total_delta:.2f} |\n"
            f"| Portfolio vega | {total_vega:.2f} |\n"
            f"| Avg PoP | {buy_df['pop'].mean():.0%} |\n"
            f"| Daily theta burn | ${total_theta:+.2f} |\n"
            f"| Weekly theta burn | ${weekly_theta:+.2f} |\n"
            f"| Positions | {len(buy_df)} contracts, {buy_df['ticker'].nunique()} tickers |"
        )
    )

### Risk at a Glance

| Metric | Value |
|---|---|
| Deployed | $13,725 of $15,000 (92%) |
| Cash reserve | $1,275 |
| **Max loss** | **$13,725** (all premiums) |
| Portfolio beta | 1.08 |
| Portfolio delta | 2.32 |
| Portfolio vega | 2.36 |
| Avg PoP | 35% |
| Daily theta burn | $-2.37 |
| Weekly theta burn | $-16.61 |
| Positions | 4 contracts, 3 tickers |

In [ ]:
# --- Export ---
export_frames: dict[str, pd.DataFrame] = {}

if fan_chains:
    export_frames["fan_chains"] = pd.concat(fan_chains.values(), ignore_index=True)
if fans:
    export_frames["best_fans"] = pd.concat(fans.values(), ignore_index=True)
if "fan_summary_df" in dir() and not fan_summary_df.empty:
    export_frames["fan_summary"] = fan_summary_df
if "buy_df" in dir() and not buy_df.empty:
    export_frames["buy_list"] = buy_df
if "pool" in dir() and not pool.empty:
    export_frames["candidate_pool"] = pool

if export_frames:
    bundle = export_report_bundle(
        prefix="call_fan",
        run_stamp=RUN_STAMP,
        output_dir=OUTPUT_DIR,
        frames=export_frames,
        metadata={
            "notebook": "call_fan_discovery",
            "tickers": list(FAN_TICKERS.keys()),
            "max_legs_per_fan": MAX_LEGS_PER_FAN,
            "starting_balance": STARTING_BALANCE,
            "total_deployed": float(total_deployed) if "total_deployed" in dir() else 0,
        },
        include_excel=EXPORT_EXCEL,
        include_zip=EXPORT_ZIP,
    )
    display(Markdown(f"**Exported to:** `{bundle['run_dir']}`"))
else:
    display(Markdown("Nothing to export — no data to save."))

**Exported to:** `outputs/call_fan_20260208_181209`